In [1]:
from __future__ import print_function
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import RMSprop
from keras.utils.data_utils import get_file
import numpy as np
import random
import sys
import io
import os

Using TensorFlow backend.


# Семинар - Генерация текста

## 1. Подготовка
## 1.1. Выбор корпуса

Сегодня мы разберем принципы работы с рекурентными сетями с использованием фреймфорка Keras. 

Вашему Вниманию, представлено не сколько вариантов текста:

In [2]:
directory = 'txt/'
files = os.listdir(directory)
files

['brodskiy.txt', 'potter.txt', 'pushkin.txt', 'potter_full.txt']

Выберите понравившийся текст: 

In [3]:
corpus = directory + files[2]
corpus

'txt/pushkin.txt'

## 1.2. Предобработка текста

In [4]:
with io.open(corpus, encoding='utf-8') as f:
    text = f.read().lower()
print('Длина корпуса:', len(text))

Длина корпуса: 66090


Посмотрим на часть текста:

In [5]:
print(text[:100])

о сколько нам открытий чудных
готовят  –  просвещенья дух
и опыт, сын ошибок трудных,
и гений, парад


Сформируйте последовательности длинны `maxlen` со сдвигом `step`, следующим образом: 
- В переменную `sentences` включите последовательность
- `next_chars` список символов который является следующим для указанной последовательности

In [6]:
maxlen = 70
step = 3
sentences = [text[i: i + maxlen] for i in range(0, len(text) - maxlen, step)]
next_chars = [text[i + maxlen] for i in range(0, len(text) - maxlen, step)]

In [7]:
print('Колличество послеовательностей:', len(sentences))

Колличество послеовательностей: 22007


Посмотрим на случайную пару: Последовательность/ответ

In [8]:
check_ran = np.random.randint(0,len(sentences))
print('После строки:')
print(sentences[check_ran])
print('')
print('Ожидаем: ',next_chars[check_ran])

После строки:
яжко мне твое явленье,
какое томное волненье
в моей душе, в моей крови

Ожидаем:  .


## 1.3. Векторизация
Далее, нам необходимо векторизвать каждую букву, для этого воспользуемся обычным Bag of chars:

In [9]:
chars = sorted(list(set(text))) #Составив список всех символов
print('Всего символов:', len(chars))
# Составьте два словаря
# Первый: список букв:номера индексов
# Второй: номера индексов:список букв

# Ваш код здесь

char_indices = {token: idx for idx, token in enumerate(np.unique(list(text)))}
indices_char = {idx: token for idx, token in enumerate(np.unique(list(text)))}

Всего символов: 61


Составьте обучающую выборку, где:  
- x - это закодированные последовательности, размерностью: (количество объектов, длинна последовательности, колличество уникальных букв), таким образом каждой букве в последовательности будет соответствовать вектор
- y - это закодированные ответы, размерностью: (количество объектов, колличество уникальных букв)

In [10]:
import gc
gc.collect()

20

In [11]:
print('Векторизация...', )
x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.integer)
y = np.zeros((len(sentences), len(chars)), dtype=np.integer)


# Ваш код здесь
for i, seq in enumerate(sentences):
    for j, token in enumerate(seq):
        x[i,j,char_indices[token]] = 1
    y[i, char_indices[next_chars[i]]] = 1
        

print('... Готово')

Векторизация...
... Готово


#### Проверим, размерность:

In [12]:
x.shape, y.shape

((22007, 70, 61), (22007, 61))

# 2. Подготовка к генерации текста
Создадим функцию `generating_text`, которая будет генерировать объекты на основании моделей:  

In [13]:
def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

def generating_text(diversity, model):
    print()
    print('...Генерация текста. Температура: ', diversity)
    start_index = random.randint(0, len(text) - maxlen - 1)
    generated = ''
    sentence = text[start_index: start_index + maxlen]
    generated += sentence
    sys.stdout.write(generated)
    for i in range(700):
        x_pred = np.zeros((1, maxlen, len(chars)))
        for t, char in enumerate(sentence):
            x_pred[0, t, char_indices[char]] = 1.

        preds = model.predict(x_pred, verbose=0)[0] #Вектор вероятностей, полученный с помощью модели
        next_index = sample(preds, diversity) # Выбираем индекс
        next_char = indices_char[next_index] #Выбираем символ из словаря 

        generated += next_char
        sentence = sentence[1:] + next_char

        sys.stdout.write(next_char)
        sys.stdout.flush()
    print()

> Temperature. We can also play with the temperature of the Softmax during sampling. Decreasing the temperature from 1 to some lower number (e.g. 0.5) makes the RNN more confident, but also more conservative in its samples. Conversely, higher temperatures will give more diversity but at cost of more mistakes (e.g. spelling mistakes, etc). In particular, setting temperature very near zero will give the most likely thing that Paul Graham might say:
>> “is that they were all the same thing that was a startup is that they were all the same thing that was a startup is that they were all the same thing that was a startup is that they were all the same”

> looks like we’ve reached an infinite loop about startups.

Источник: [The Unreasonable Effectiveness of Recurrent Neural Networks](http://karpathy.github.io/2015/05/21/rnn-effectiveness/)

# 3. Обучение моделей

In [23]:
from keras.layers import SimpleRNN, GRU, LSTM, Dropout

## 3.1. VanilaRNN
Создадим простую RNN с одним слоем из 128 нейронов

In [15]:
model = Sequential()
model.add(SimpleRNN(128, input_shape=(maxlen, len(chars))))
model.add(Dense(len(chars), activation='softmax'))

optimizer = RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)
model.summary()

Instructions for updating:
Colocations handled automatically by placer.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
simple_rnn_1 (SimpleRNN)     (None, 128)               24320     
_________________________________________________________________
dense_1 (Dense)              (None, 61)                7869      
Total params: 32,189
Trainable params: 32,189
Non-trainable params: 0
_________________________________________________________________


In [16]:
model.fit(x, y, batch_size=128,
          epochs=20, validation_split=0.2)

Instructions for updating:
Use tf.cast instead.
Train on 17605 samples, validate on 4402 samples
Epoch 1/20
17605/17605 [==============================] - 8s 445us/step - loss: 3.6717 - val_loss: 3.5305
Epoch 2/20
17605/17605 [==============================] - 10s 578us/step - loss: 3.4212 - val_loss: 3.3776
Epoch 3/20
17605/17605 [==============================] - 12s 671us/step - loss: 3.0851 - val_loss: 3.0867
Epoch 4/20
17605/17605 [==============================] - 8s 473us/step - loss: 2.9948 - val_loss: 3.2440
Epoch 5/20
17605/17605 [==============================] - 11s 598us/step - loss: 2.9885 - val_loss: 2.9403
Epoch 6/20
17605/17605 [==============================] - 14s 815us/step - loss: 2.8811 - val_loss: 2.9355
Epoch 7/20
17605/17605 [==============================] - 11s 610us/step - loss: 2.8707 - val_loss: 2.9521
Epoch 8/20
17605/17605 [==============================] - 10s 565us/step - loss: 2.8665 - val_loss: 2.9181
Epoch 9/20
17605/17605 [=========================

#### Сгенерируем текст:

In [24]:
generating_text(0.5, model)


...Генерация текста. Температура:  0.5
емит правдивая хвала:
кто славил марса и темиру
и бранную повесил лиру                 

/home/keleas/anaconda3/envs/keras/lib/python3.7/site-packages/ipykernel_launcher.py:4: RuntimeWarning: invalid value encountered in log
  after removing the cwd from sys.path.


Поэкспериментируйте с параметром `diversity` и посмотрите как изменяется генерируемый текст

## 3.2. Need more layers... 
Добавьте второй слой в нашу RNN, обучите и посмотрите на результат

In [25]:
model = Sequential()
model.add(SimpleRNN(256, input_shape=(maxlen, len(chars))))

model.add(Dense(len(chars), activation='softmax'))

optimizer = RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
simple_rnn_2 (SimpleRNN)     (None, 256)               81408     
_________________________________________________________________
dense_2 (Dense)              (None, 61)                15677     
Total params: 97,085
Trainable params: 97,085
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.fit(x, y, batch_size=128,
          epochs=10, validation_split=0.2)

In [ ]:
generating_text(0.5, model)

Проверьте какой текст получается при генерации для разного параметра `diversity`

In [ ]:
generating_text(0.8, model)

In [ ]:
generating_text(1.5, model)

# 3.3 Больше экспериментов
Попробуйте различные варианты сетей: GRU и LSTM. Сравните результаты и выберите удачные примеры генерации текста. 

In [37]:
model = Sequential()
model.add(LSTM(128, return_sequences=True, input_shape=(maxlen, len(chars))))
model.add(LSTM(128))
model.add(Dense(len(chars), activation='softmax'))

optimizer = RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_11 (LSTM)               (None, 70, 128)           97280     
_________________________________________________________________
lstm_12 (LSTM)               (None, 128)               131584    
_________________________________________________________________
dense_6 (Dense)              (None, 61)                7869      
Total params: 236,733
Trainable params: 236,733
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.fit(x, y, batch_size=128,
          epochs=5, validation_split=0.2)

Train on 17605 samples, validate on 4402 samples
Epoch 1/5
17605/17605 [==============================] - 41s 2ms/step - loss: 3.3957 - val_loss: 3.3469
Epoch 2/5
17605/17605 [==============================] - 41s 2ms/step - loss: 3.3521 - val_loss: 3.4641
Epoch 3/5
17605/17605 [==============================] - 43s 2ms/step - loss: 3.3574 - val_loss: 3.3852
Epoch 4/5
17605/17605 [==============================] - 43s 2ms/step - loss: 3.3583 - val_loss: 3.4391
Epoch 5/5
11136/17605 [=================>............] - ETA: 14s - loss: 3.3594

In [32]:
generating_text(0.8, model)


...Генерация текста. Температура:  0.8
ывало тешил ханов
стихов гремучим жемчугом.

на нити праздного веселья,
ны лю, внем уж честьки росостые,
взем с чут? моей надожик,
друг как мого гружа прокучанья
я в, подлость бериструут рескаел.
и тет берспоего стун
ей ролон забера темый
на редостомпо устыл и чаляв,
ком мое глобил муты , воценья
меня в горша далан,
своей горот шубный
расной
понька, бул ул всон сторлет савы,
и твой ль перя, не ует половил.

восториталь судькой поло снил
не узавстим насможит, над вилиты,
вотого веждет поленья и выл,
смате сель тире полавый бор
пырел поустал поросей восторши,
ужа и бесто верской
дороды утрыни сер,
и лебя злил сторой ном,
ут мине не лостенья
и сыны гразной разна,
вы пералу морк хологу волчавный небю
е раззлоной,
и берег в не  путь глеком зизоком
кела гредил с тоб
